In [ ]:
import pandas as pd
import numpy as np

# Load data from colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
def get_embeddings(text):
  tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
  with torch.no_grad():
    output = model(**tokens)
    embeddings = output.last_hidden_state
  return embeddings

In [ ]:
data = pd.read_csv('/content/drive/My Drive/BT4012 Group 06/Data/feature_engineering_data_new.csv')
data.columns

Index(['job_id', 'title', 'department', 'salary_range', 'company_profile',
       'description', 'requirements', 'benefits', 'telecommuting',
       'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent', 'min_salary', 'max_salary', 'noise_removed_title',
       'noise_removed_company_profile', 'noise_removed_description',
       'noise_removed_requirements', 'noise_removed_benefits',
       'tokenized_title', 'pos_tags_title', 'tokenized_company_profile',
       'pos_tags_company_profile', 'tokenized_description',
       'pos_tags_description', 'tokenized_requirements',
       'pos_tags_requirements', 'tokenized_benefits', 'pos_tags_benefits',
       'country', 'province', 'city', 'length of profile name',
       'length of description', 'length of requirements', 'length of benefits',
       'capital_words_number_profile', 'capital_words_number_description',
       'capital_words_number_re

In [ ]:
pd.set_option('display.max_columns', None)
data.head()

,job_id,title,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,min_salary,max_salary,noise_removed_title,noise_removed_company_profile,noise_removed_description,noise_removed_requirements,noise_removed_benefits,tokenized_title,pos_tags_title,tokenized_company_profile,pos_tags_company_profile,tokenized_description,pos_tags_description,tokenized_requirements,pos_tags_requirements,tokenized_benefits,pos_tags_benefits,country,province,city,length of profile name,length of description,length of requirements,length of benefits,capital_words_number_profile,capital_words_number_description,capital_words_number_requirements,capital_words_number_benefits,number of sentences_profile,number of sentences_description,number of sentences_requirements,number of sentences_benefits,number of stopwords_profile_description,number of stopwords_profile,number of stopwords_requirements,number of stopwords_benefit,sentiment score_profile,sentiment score_requirement,sentiment score_benefits,sentiment score_description,embeddings_profile
0,1,Marketing Intern,Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,unknown,0,1,0,Other,Internship,Bachelor's Degree,Information Technology and Services,Marketing,0,35000.0,50000.0,Marketing Intern,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,unknown,"['marketing', 'intern']","['PROPN', 'PROPN']","['food52', 'create', 'groundbreaking', 'award'...","['ADJ', 'VERB', 'NOUN', 'NOUN', 'VERB', 'NOUN'...","['food52', 'fast', 'grow', 'james', 'beard', '...","['NOUN', 'ADV', 'VERB', 'PROPN', 'PROPN', 'PRO...","['experience', 'content', 'management', 'syste...","['NOUN', 'NOUN', 'NOUN', 'NOUN', 'ADJ', 'CCONJ...",['unknown'],['ADJ'],US,NY,New York,885,905,852,7,2,1,0,0,5,1,2,1,45,49,42,0,0.9856,0.7345,0.0000,0.6486,"tensor([[[-0.2207, -0.1446, -0.2817, ..., 0...."
1,2,Customer Service - Cloud Video Production,Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,Bachelor's Degree,Marketing and Advertising,Customer Service,0,35000.0,50000.0,Customer Service - Cloud Video Production,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,"['customer', 'service', 'cloud', 'video', 'pro...","['PROPN', 'PROPN', 'PROPN', 'PROPN', 'PROPN']","['90', 'seconds', 'world', 'cloud', 'video', '...","['NUM', 'PROPN', 'NOUN', 'PROPN', 'PROPN', 'PR...","['organise', 'focused', 'vibrant', 'awesome!do...","['VERB', 'ADJ', 'PROPN', 'PROPN', 'NOUN', 'NOU...","['expect', 'key', 'responsibility', 'communica...","['VERB', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'NUM',...","['usthrough', '90', 'seconds', 'team', 'gain',...","['ADP', 'NUM', 'PROPN', 'NOUN', 'VERB', 'NOUN'...",NZ,,Auckland,1286,2077,1433,1292,0,1,0,3,4,15,2,5,104,43,68,112,0.9313,0.9929,0.9716,0.9951,"tensor([[[-0.3885, -0.1363, -0.0421, ..., -0...."
2,3,Commissioning Machinery Assistant (CMA),Sales,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,unknown,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,0,35000.0,50000.0,Commissioning Machinery Assistant (CMA),Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,unknown,"['commis

In [ ]:
processed_data = data[['job_id', 'department', 'salary_range', 'company_profile',	'description',	'requirements',	'benefits', 'telecommuting',
       'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent', 'min_salary', 'max_salary',
       'tokenized_title', 'pos_tags_title', 'tokenized_company_profile',
       'pos_tags_company_profile', 'tokenized_description',
       'pos_tags_description', 'tokenized_requirements',
       'pos_tags_requirements', 'tokenized_benefits', 'pos_tags_benefits',
       'country', 'province', 'city', 'sentiment score_profile',
       'sentiment score_requirement', 'sentiment score_benefits',
       'sentiment score_description', 'embeddings_profile']]

In [ ]:
processed_data.head()

,job_id,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,min_salary,max_salary,tokenized_title,pos_tags_title,tokenized_company_profile,pos_tags_company_profile,tokenized_description,pos_tags_description,tokenized_requirements,pos_tags_requirements,tokenized_benefits,pos_tags_benefits,country,province,city,sentiment score_profile,sentiment score_requirement,sentiment score_benefits,sentiment score_description,embeddings_profile
0,1,Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,unknown,0,1,0,Other,Internship,Bachelor's Degree,Information Technology and Services,Marketing,0,35000.0,50000.0,"['marketing', 'intern']","['PROPN', 'PROPN']","['food52', 'create', 'groundbreaking', 'award'...","['ADJ', 'VERB', 'NOUN', 'NOUN', 'VERB', 'NOUN'...","['food52', 'fast', 'grow', 'james', 'beard', '...","['NOUN', 'ADV', 'VERB', 'PROPN', 'PROPN', 'PRO...","['experience', 'content', 'management', 'syste...","['NOUN', 'NOUN', 'NOUN', 'NOUN', 'ADJ', 'CCONJ...",['unknown'],['ADJ'],US,NY,New York,0.9856,0.7345,0.0000,0.6486,"tensor([[[-0.2207, -0.1446, -0.2817, ..., 0...."
1,2,Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,Bachelor's Degree,Marketing and Advertising,Customer Service,0,35000.0,50000.0,"['customer', 'service', 'cloud', 'video', 'pro...","['PROPN', 'PROPN', 'PROPN', 'PROPN', 'PROPN']","['90', 'seconds', 'world', 'cloud', 'video', '...","['NUM', 'PROPN', 'NOUN', 'PROPN', 'PROPN', 'PR...","['organise', 'focused', 'vibrant', 'awesome!do...","['VERB', 'ADJ', 'PROPN', 'PROPN', 'NOUN', 'NOU...","['expect', 'key', 'responsibility', 'communica...","['VERB', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'NUM',...","['usthrough', '90', 'seconds', 'team', 'gain',...","['ADP', 'NUM', 'PROPN', 'NOUN', 'VERB', 'NOUN'...",NZ,,Auckland,0.9313,0.9929,0.9716,0.9951,"tensor([[[-0.3885, -0.1363, -0.0421, ..., -0...."
2,3,Sales,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,unknown,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,0,35000.0,50000.0,"['commission', 'machinery', 'assistant', 'cma']","['VERB', 'PROPN', 'PROPN', 'PROPN']","['valor', 'services', 'provide', 'workforce', ...","['PROPN', 'PROPN', 'VERB', 'PROPN', 'PROPN', '...","['client', 'locate', 'houston', 'actively', 's...","['NOUN', 'VERB', 'PROPN', 'ADV', 'VERB', 'ADJ'...","['implement', 'pre', 'commissioning', 'commiss...","['VERB', 'ADJ', 'ADJ', 'VERB', 'NOUN', 'ADJ', ...",['unknown'],['ADJ'],US,IA,Wever,0.9618,0.9260,0.0000,0.9509,"tensor([[[-8.7042e-03, 3.5888e-02, -4.8977e-0..."
3,4,Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,35000.0,50000.0,"['account', 'executive', 'washington', 'dc']","['PROPN', 'PROPN', 'PROPN', 'PROPN']","['passion', 'improve', 'quality', 'life', 'geo...","['NOUN', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN...","['company', 'esri', '–', 'environmental', 'sys...","['PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PROPN', ...","['education', 'bachelor', 'master', 'gis', 'bu...","['NOUN', 'SPACE', 'PROPN', 'PROPN', 'PROPN', '...","['culture', 'corporate', '—', 'collaborative',...","['NOUN', 'ADJ', 'PUNCT', 'ADJ', 'ADJ', 'NOUN',...",US,DC,Washington,0.9612,0.9819,0.8462,0.9957,"tensor([[[-0.2483, 0.2116, 0.1013, ..., -0...."
4,5

In [ ]:
processed_data['department'].nunique()

1337

In [ ]:
X = processed_data[['job_id', 'department', 'salary_range', 'company_profile',	'description',	'requirements',	'benefits', 'telecommuting',
       'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent','min_salary', 'max_salary',
       'tokenized_title', 'pos_tags_title', 'tokenized_company_profile',
       'pos_tags_company_profile', 'tokenized_description',
       'pos_tags_description', 'tokenized_requirements',
       'pos_tags_requirements', 'tokenized_benefits', 'pos_tags_benefits',
       'country', 'province', 'city', 'sentiment score_profile',
       'sentiment score_requirement', 'sentiment score_benefits',
       'sentiment score_description', 'embeddings_profile']]
y= df[[]]
# Split the data into training and testing sets with a fixed random_state
train, test, train_2, test_2 = train_test_split(X, y, test_size=0.2, random_state=42)

# Create new columns to label rows
processed_data['data_split'] = 'Training'  # Set 'Training' for all rows
processed_data.loc[processed_data.index.isin(test.index), 'data_split'] = 'Testing'  # Set 'Testing' for test set rows

<ipython-input-20-e5f53a9d7b9e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['data_split'] = 'Training'  # Set 'Training' for all rows


In [ ]:
# # department - frequency & target encoding
# # frequency encoding
# department_frequencies = processed_data['department'].value_counts(normalize=True)
# processed_data['department_frequency'] = processed_data['department'].map(department_frequencies)

# # target encoding (mean encoding)
# department_means = processed_data.groupby('department')['fraudulent'].mean()
# processed_data['department_mean_target'] = processed_data['department'].map(department_means)
# processed_data.head()

# Frequency encoding for training data
train_department_frequencies = train['department'].value_counts(normalize=True)
train['department_frequency'] = train['department'].map(train_department_frequencies)

# Mean target encoding for training data
train_department_means = train.groupby('department')['fraudulent'].mean()
train['department_mean_target'] = train['department'].map(train_department_means)

# Apply the same encoding to the testing data
test['department_frequency'] = test['department'].map(train_department_frequencies)  # Use the frequencies from the training data
test['department_mean_target'] = test['department'].map(train_department_means)  # Use the means from the training data

# Reindex the dataframes to maintain the original index sequence
train_data = train.sort_index()
test_data = test.sort_index()

# Combine the encoded training and testing datasets
encoded_data = pd.concat([train_data, test_data]).sort_index()

In [ ]:
processed_data = encoded_data

In [ ]:
# description - Word2Vec
from gensim.models import Word2Vec
# from nltk.tokenize import word_tokenize
# import nltk
# nltk.download('punkt')

# processed_data['tokenized_description'] = processed_data['description'].apply(word_tokenize)
word2vec_model = Word2Vec(sentences=processed_data['tokenized_description'], vector_size=100, window=5, min_count=1)

word2vec_model.save('word2vec_model.bin')

# Load the model
# word2vec_model = Word2Vec.load('word2vec_model.bin')

def get_mean_embedding(tokens, model):
    valid_tokens = [token for token in tokens if token in model.wv]
    if not valid_tokens:
        return np.zeros(model.vector_size)
    embeddings = [model.wv[token] for token in valid_tokens]
    mean_embedding = np.mean(embeddings, axis=0)
    return mean_embedding

processed_data['mean_embeddings_description'] = processed_data['tokenized_description'].apply(lambda x: get_mean_embedding(x, word2vec_model))

processed_data.head()



In [ ]:
processed_data

,job_id,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,min_salary,max_salary,tokenized_title,pos_tags_title,tokenized_company_profile,pos_tags_company_profile,tokenized_description,pos_tags_description,tokenized_requirements,pos_tags_requirements,tokenized_benefits,pos_tags_benefits,country,province,city,sentiment score_profile,sentiment score_requirement,sentiment score_benefits,sentiment score_description,embeddings_profile,department_frequency,department_mean_target,mean_embeddings_description
0,1,Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,unknown,0,1,0,Other,Internship,Bachelor's Degree,Information Technology and Services,Marketing,0,35000.0,50000.0,"['marketing', 'intern']","['PROPN', 'PROPN']","['food52', 'create', 'groundbreaking', 'award'...","['ADJ', 'VERB', 'NOUN', 'NOUN', 'VERB', 'NOUN'...","['food52', 'fast', 'grow', 'james', 'beard', '...","['NOUN', 'ADV', 'VERB', 'PROPN', 'PROPN', 'PRO...","['experience', 'content', 'management', 'syste...","['NOUN', 'NOUN', 'NOUN', 'NOUN', 'ADJ', 'CCONJ...",['unknown'],['ADJ'],US,NY,New York,0.9856,0.7345,0.0000,0.6486,"tensor([[[-0.2207, -0.1446, -0.2817, ..., 0....",0.022301,0.006270,[ 0.49690506 -0.13427249 0.5732839 0.082606...
1,2,Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,Bachelor's Degree,Marketing and Advertising,Customer Service,0,35000.0,50000.0,"['customer', 'service', 'cloud', 'video', 'pro...","['PROPN', 'PROPN', 'PROPN', 'PROPN', 'PROPN']","['90', 'seconds', 'world', 'cloud', 'video', '...","['NUM', 'PROPN', 'NOUN', 'PROPN', 'PROPN', 'PR...","['organise', 'focused', 'vibrant', 'awesome!do...","['VERB', 'ADJ', 'PROPN', 'PROPN', 'NOUN', 'NOU...","['expect', 'key', 'responsibility', 'communica...","['VERB', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'NUM',...","['usthrough', '90', 'seconds', 'team', 'gain',...","['ADP', 'NUM', 'PROPN', 'NOUN', 'VERB', 'NOUN'...",NZ,,Auckland,0.9313,0.9929,0.9716,0.9951,"tensor([[[-0.3885, -0.1363, -0.0421, ..., -0....",0.000280,0.000000,[ 0.47468713 -0.15353158 0.5474723 0.045763...
2,3,Sales,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,unknown,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,0,35000.0,50000.0,"['commission', 'machinery', 'assistant', 'cma']","['VERB', 'PROPN', 'PROPN', 'PROPN']","['valor', 'services', 'provide', 'workforce', ...","['PROPN', 'PROPN', 'VERB', 'PROPN', 'PROPN', '...","['client', 'locate', 'houston', 'actively', 's...","['NOUN', 'VERB', 'PROPN', 'ADV', 'VERB', 'ADJ'...","['implement', 'pre', 'commissioning', 'commiss...","['VERB', 'ADJ', 'ADJ', 'VERB', 'NOUN', 'ADJ', ...",['unknown'],['ADJ'],US,IA,Wever,0.9618,0.9260,0.0000,0.9509,"tensor([[[-8.7042e-03, 3.5888e-02, -4.8977e-0...",0.675685,0.044594,[ 0.50534314 -0.11600273 0.59066945 0.123568...
3,4,Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,35000.0,50000.0,"['account', 'executive', 'washington', 'dc']","['PROPN', 'PROPN', 'PROPN', 'PROPN']","['passion', 'improve', 'quality', 'life', 'geo...","['NOUN', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN...","['company', 'esri', '–', 'environmental', 'sys...","['PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PROPN', ...","['education', 'bachelor', 'master', 

In [ ]:
processed_data.to_csv('/content/drive/My Drive/BT4012 Group 06/Data/feature_engineering_data_v2.csv', index=False)